<a href="https://colab.research.google.com/github/akbism/COVID-QA/blob/main/ModelEvaluation/7_Evaluate_single_model_roberta_squad2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script is for evaluation of a model or checkpoint.

# Setting up the google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/Colab\ Notebooks/LJMU/covidqa/biobert-pytorch/question-answering/20210711
!pip install tqdm==4.41.1
!pip install transformers datasets
# #!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl
# VERSION = "1.8.1"
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

Mounted at /content/gdrive
[Errno 2] No such file or directory: 'gdrive/My Drive/Colab Notebooks/LJMU/covidqa/biobert-pytorch/question-answering/20210711'
/content
     |████████████████████████████████| 2.6 MB 8.4 MB/s 
     |████████████████████████████████| 542 kB 35.6 MB/s 
     |████████████████████████████████| 3.3 MB 40.0 MB/s 
     |████████████████████████████████| 895 kB 33.3 MB/s 
     |████████████████████████████████| 636 kB 41.9 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 243 kB 43.3 MB/s 
     |████████████████████████████████| 118 kB 51.4 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Setting the evaluation parameters

In [ ]:
import datasets, collections
import numpy as np
from transformers import Trainer, TrainingArguments
from datasets import load_from_disk, load_metric, load_dataset
from transformers import AutoModelForQuestionAnswering, AutoTokenizer 
from transformers.data.metrics.squad_metrics import squad_evaluate

class eval_Args:
  data_dir = '/content/gdrive/MyDrive/Colab Notebooks/LJMU/covidqa/biobert-pytorch/datasets/QA/SQuAD'  #'/content/gdrive/MyDrive/Colab Notebooks/LJMU/covidqa/biobert-pytorch/datasets/QA/BioASQ/BioASQ-678b'
  train_file = 'train-v2.0_modified.json' # 'BioASQ-train-factoid-6_8b-full-annotated_modified.json' #  
  predict_file = 'dev-v2.0_modified.json' #  'COVID-QA-Modified-Modified.json' # 'BioASQ-train-factoid-7b-full-annotated_modified.json'
  model_name = 'deepset/roberta-base-squad2'#'/content/gdrive/MyDrive/Colab Notebooks/LJMU/covidqa/experiments/case_1_1_2/TPU/result_bkp/checkpoint-610' 
  max_length = 384 # The maximum length of a feature (question and context)
  batch_size = 16
  doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
  squad_v2 = True
  tokenizer_name = 'roberta-base'
  return_overflowing_tokens=True,
  return_offsets_mapping=True,
  padding="max_length"
  n_best_size = 20
  max_answer_length = 30

eval_args=eval_Args()


tokenizer = AutoTokenizer.from_pretrained(eval_args.tokenizer_name)
pad_on_right = tokenizer.padding_side == "right"

training_args = TrainingArguments(
output_dir='./results',
do_eval = True)


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

# Defining important functions

In [ ]:
import os
import datasets
from tqdm.auto import tqdm
import pandas as pd

def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    args=eval_Args()
    # import pdb
    # pdb.set_trace()
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=args.max_length,
        stride=args.doc_stride,
        return_overflowing_tokens=args.return_overflowing_tokens,
        return_offsets_mapping=args.return_offsets_mapping,
        padding=args.padding,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


def postprocess_qa_predictions(args, examples, features, raw_predictions):
    n_best_size = args.n_best_size
    max_answer_length = args.max_answer_length
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not args.squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

def format_and_score(args, qa_df, final_predictions): 
  # import pdb
  # pdb.set_trace()
  metric = load_metric("squad_v2" if args.squad_v2 else "squad")
  if args.squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
  else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
  references = [{"id": ex["id"], "answers": ex["answers"]} for ex in qa_df]
  return metric.compute(predictions=formatted_predictions, references=references)

def get_eval_metrics():

  args=eval_Args()
  val_input_filename = os.path.join(args.data_dir, args.predict_file)
  if args.train_file is None:
    train_input_filename = val_input_filename
  else:
    train_input_filename = os.path.join(args.data_dir, args.train_file)

  print("*** Reading Data ***")
  qa_data = load_dataset("json", data_files={"train": train_input_filename, "validation": val_input_filename} )

  qa_data = qa_data.filter(lambda example, indice: indice != 107709, with_indices=True)
  # qa_data['train']=qa_data['train'].select(range(2_000)) 
  # qa_data['validation']=qa_data['validation'].select(range(1_000)) 

  qa_data_features = qa_data.map(
      prepare_validation_features,
      batched=True,
      remove_columns=qa_data["validation"].column_names)
  #### squad v1 will give error 
  model_name=args.model_name
  # result= pd.DataFrame( columns=['exact', 'f1', 'total', 'HasAns_exact', 'HasAns_f1', 'HasAns_total', 'best_exact', 'best_exact_thresh', 'best_f1', 'best_f1_thresh', 'model_name', 'split'])
  result= pd.DataFrame( columns=['exact', 'f1', 'total', 'HasAns_exact', 'HasAns_f1', 'HasAns_total','NoAns_exact','NoAns_f1','NoAns_total', 'best_exact', 'best_exact_thresh', 'best_f1', 'best_f1_thresh', 'model_name', 'split'])
  model = AutoModelForQuestionAnswering.from_pretrained(model_name)
  trainer = Trainer(
  model=model,
  args=training_args)

  print("*** Prediction, postprocessing and formatting ***")
  # import pdb
  # pdb.set_trace()
  val_raw_predictions = trainer.predict(qa_data_features['validation'])
  val_final_predictions = postprocess_qa_predictions(args, qa_data["validation"], qa_data_features['validation'], val_raw_predictions.predictions)
  val_score=format_and_score(args, qa_data["validation"], val_final_predictions)  
  # import pdb
  # pdb.set_trace()
  if args.train_file is None:
    train_score = val_score
  else:
    train_raw_predictions = trainer.predict(qa_data_features['train'])
    train_final_predictions = postprocess_qa_predictions(args,qa_data["train"], qa_data_features['train'], train_raw_predictions.predictions)
    train_score=format_and_score(args, qa_data["train"],  train_final_predictions)  
  
  val_score['model_name'] = args.model_name
  val_score['split'] ='validation'
  train_score['model_name'] =args.model_name
  train_score['split'] ='train'
  # print(checkpoint)
  # import pdb
  # pdb.set_trace()
  temp=pd.concat([pd.DataFrame.from_dict(val_score, orient='index').T, pd.DataFrame.from_dict(train_score, orient='index').T]).reset_index()
  result=result.append(temp)
  return result

#Evaluation of the model / checkpoint

In [ ]:
temp1 = get_eval_metrics()

*** Reading Data ***


Using custom data configuration default-11466adaa44727ec


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-11466adaa44727ec/0.0.0/45636811569ec4a6630521c18235dfbbab83b7ab572e3393c5ba68ccabe98264. Subsequent calls will reuse this data.


...

Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

The following columns in the test set  don't have a corresponding argument in `RobertaForQuestionAnswering.forward` and have been ignored: example_id, offset_mapping.
***** Running Prediction *****
  Num examples = 131822
  Batch size = 8


Post-processing 130318 example predictions split into 131822 features.


  0%|          | 0/130318 [00:00<?, ?it/s]

In [ ]:
temp1

,exact,f1,total,HasAns_exact,HasAns_f1,HasAns_total,NoAns_exact,NoAns_f1,NoAns_total,best_exact,best_exact_thresh,best_f1,best_f1_thresh,model_name,split,index
0,79.9208,82.9175,11873,77.9352,83.9372,5928,81.9008,81.9008,5945,79.9208,0,82.9175,0,deepset/roberta-base-squad2,validation,0.0
1,80.6696,88.2182,130318,75.1555,86.486,86820,91.6755,91.6755,43498,80.6819,0,88.2304,0,deepset/roberta-base-squad2,train,0.0
